# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [53]:
import pandas as pd
import pymongo

In [54]:
client = pymongo.MongoClient('127.0.0.1', 27017)
database = client['exercices']
collection = database['youtube']

In [55]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [56]:
df_youtube_noNaN = df_youtube.dropna()

### Importer les données

In [57]:
df = df_youtube_noNaN.to_dict(orient='records')
collection.insert_many(df)

## Question 1  

In [58]:
cur = collection.find({"channel_title" : "Apple"})
list(cur)

[{'_id': ObjectId('601b0086216f59ef531942de'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 7860119,
  'likes': 185853,
  'dislikes': 26679,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 13.09},
 {'_id': ObjectId('601b0086216f59ef531943a8'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 12200526,
  'likes': 258842,
  'dislikes': 44339,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 14.09},
 {'_id': Obje

## Question 2

In [59]:
cur = collection.aggregate([{"$group" : {"_id" : "$category_id", "Nombre" : {"$sum" : 1}}}])
len(list(cur))

16

## Question 3

In [60]:
cur = collection.find()
for element in cur:
    collection.update_one(element,{'$set':{'tags':element['tags'].split('|')}})
collection.find_one()

{'_id': ObjectId('601b0086216f59ef531942dd'),
 'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': ['logan paul vlog',
  'logan paul',
  'logan',
  'paul',
  'olympics',
  'logan paul youtube',
  'vlog',
  'daily',
  'comedy',
  'hollywood',
  'parrot',
  'maverick',
  'bird',
  'maverick clothes',
  'diamond play button',
  'logan paul diamond play button',
  '10M subscribers',
  'logan paul 1 year vlogging',
  '1 year vlog',
  'dwarf mamba play button',
  'logan paul history',
  'youtube history',
  '10M',
  '10M plaque',
  'youtube button',
  'diamond button',
  'logang',
  'logang 4 life'],
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': 13.09}

## Question 4

In [61]:
# Les 5 vidéos les plus vues.
cur = collection.find().sort([("views", -1)])
next(cur)

{'_id': ObjectId('601b0086216f59ef53194a50'),
 'video_id': 'MBdVXkSdhwU',
 'title': "BTS (방탄소년단) 'DNA' Official MV",
 'channel_title': 'ibighit',
 'category_id': 10,
 'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
 'views': 41500672,
 'likes': 2010366,
 'dislikes': 78076,
 'comment_total': 736179,
 'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
 'date': 22.09}

## Question 5

In [50]:
cur = collection.aggregate([{"$group" : {"_id" : "$category_id", "Moyenne" : {"$avg" : "$views"}}}])
list(cur)

[{'_id': 10, 'Moyenne': 1176553.612},
 {'_id': 29, 'Moyenne': 1110334.2142857143},
 {'_id': 28, 'Moyenne': 924730.548828125},
 {'_id': 26, 'Moyenne': 537665.1806674339},
 {'_id': 25, 'Moyenne': 540955.5569823436},
 {'_id': 17, 'Moyenne': 728434.3243902439},
 {'_id': 2, 'Moyenne': 607693.8706896552},
 {'_id': 22, 'Moyenne': 971532.7573696146},
 {'_id': 23, 'Moyenne': 1240073.294039735},
 {'_id': 15, 'Moyenne': 651404.3879310344},
 {'_id': 43, 'Moyenne': 8492.5},
 {'_id': 24, 'Moyenne': 1154868.4397251718},
 {'_id': 20, 'Moyenne': 681081.8170731707},
 {'_id': 1, 'Moyenne': 1039472.6560846561},
 {'_id': 19, 'Moyenne': 464041.0833333333},
 {'_id': 27, 'Moyenne': 547582.4700598803}]

## Question 6 

In [51]:
cur = collection.aggregate([{"$group" : {"_id" : "$channel_title", "Moyenne" : {"$avg" : "$likes"}}},{"$sort":{"Moyenne":-1}}])
next(cur)

[{'_id': 'ZaynVEVO', 'Moyenne': 1431683.0},
 {'_id': 'ibighit', 'Moyenne': 1371766.25},
 {'_id': 'melanie martinez', 'Moyenne': 911871.5},
 {'_id': 'BETNetworks', 'Moyenne': 769687.625},
 {'_id': 'jypentertainment', 'Moyenne': 758826.6666666666}]